In [58]:
import json
import binascii
import os
import urllib3
import certifi

## Проверка отклика на вакансию

In [ ]:
# Get authorisation params
with open('user_token_params.json', 'r', encoding='utf-8') as f:
    user_token_params = json.load(f)

In [52]:
vacancy_id = '70867905' 
resume_id = '6e1019f2ff040603530039ed1f4b593554312' # IT
message = ''
response_params = {}

In [ ]:
#----------------------------------------------------------------------------
# https://julien.danjou.info/handling-multipart-form-data-python/
#----------------------------------------------------------------------------
def encode_multipart_formdata(fields):

    boundary = binascii.hexlify(os.urandom(16)).decode('ascii')
    body = (
        "".join("--%s\r\n"
                "Content-Disposition: form-data; name=\"%s\"\r\n"
                "\r\n"
                "%s\r\n" % (boundary, field, value)
                for field, value in fields.items()) +
        "--%s--\r\n" % boundary
    )
    content_type = "multipart/form-data; boundary=%s" % boundary
    return body, content_type

In [26]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [ ]:
body, content_type = encode_multipart_formdata({'vacancy_id': '{}'.format(vacancy_id),
                           'resume_id' : '{}'.format(resume_id),
                           'message'   : '{}'.format(message)})

r = http.request('POST', 'https://api.hh.ru/negotiations',
                                    headers = {'Authorization' : 'Bearer {}'.format(user_token_params['access_token']),
                                              'Accept'        : '*/*',
                                              'User-Agent'    : 'CIO_jbSearch (ssv.ruby@gmail.com)',
                                              'Content-Type'  : '{}'.format(content_type)},
                                    body =  '{}'.format(body)
                                  )

In [55]:
response_params['status'] = r.status
response_params['reason'] = r.reason
response_params['date'] = r.getheader('Date')
response_params['response_link'] = 'https://hh.ru/applicant{}'.format(r.getheader('Location'))
response_params

{'status': 201,
 'reason': 'Created',
 'date': 'Sat, 08 Oct 2022 14:03:16 GMT',
 'response_link': 'https://hh.ru/applicant/negotiations/2928346342'}